<a href="https://colab.research.google.com/github/akingno/colab_temp/blob/main/Mmed_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install transformers accelerate



Looking in indexes: https://download.pytorch.org/whl/cu118


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("Henrychur/MMed-Llama-3-8B")
model = AutoModelForCausalLM.from_pretrained(
    "Henrychur/MMed-Llama-3-8B",
    torch_dtype=torch.float16,  # 使用 16-bit 模式
    device_map="auto"           # 自动分配到 GPU（如果可用）
)




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/791 [00:00<?, ?B/s]

...

model-00005-of-00007.safetensors:   0%|          | 0.00/4.83G [00:00<?, ?B/s]

model-00006-of-00007.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00007-of-00007.safetensors:   0%|          | 0.00/2.57G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

In [3]:
import torch
print(torch.cuda.is_available())

True


In [4]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [5]:
!git clone https://github.com/pubmedqa/pubmedqa.git

Cloning into 'pubmedqa'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 40 (delta 0), reused 1 (delta 0), pack-reused 37 (from 1)
Receiving objects: 100% (40/40), 704.87 KiB | 8.49 MiB/s, done.
Resolving deltas: 100% (12/12), done.


In [6]:
cd pubmedqa

/content/pubmedqa


In [7]:
cd preprocess

/content/pubmedqa/preprocess


In [8]:
!python split_dataset.py pqal

In [ ]:
import json
from google.colab import files

# 读取数据集
with open("/content/pubmedqa/data/test_set.json", "r") as f:
    data = json.load(f)

results = []


keys = list(data.keys())[20:40]

for key in keys:
    input_text = data[key]["QUESTION"]  # 获取问题
    data_answer_text = data[key]["LONG_ANSWER"]  # 获取标准答案

    print(f"问题 (Key: {key}): {input_text}")
    print(f"标准答案: {data_answer_text}")

    # 创建模型输入，加入问题和“回答”提示
    prompt = f"You're a doctor, kindly address the medical query in English. Let’s think step by step. Question：{input_text}\n Your answer："

    # 输入模型
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # 生成答案
    outputs = model.generate(
        **inputs,
        max_new_tokens=128,
        do_sample=True,
        temperature=0.6,
        top_p=0.9
    )

    # 解码生成的答案
    generated_answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # 打印生成的回答和标准答案
    print(f"生成回答: {generated_answer}")
    print("-" * 50)  # 分隔符

    # 将结果添加到列表
    results.append({
        "key": key,
        "question": input_text,
        "standard_answer": data_answer_text,
        "generated_answer": generated_answer
    })

# 将结果保存到本地 JSON 文件
output_file = "/content/mmed_generated_results.json"
with open(output_file, "w") as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

# 下载保存的 JSON 文件
files.download(output_file)


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


问题 (Key: 11146778): Risk stratification in emergency surgical patients: is the APACHE II score a reliable marker of physiological impairment?
标准答案: For risk stratification in emergency surgical patients, it is essential to measure the APACHE II score before surgical treatment. Longitudinal APACHE II scoring reveals continuous improvement of the score in surviving patients but has no therapeutic relevance in the individual patient.


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


生成回答: You're a doctor, kindly address the medical query in English. Let’s think step by step. Question：Risk stratification in emergency surgical patients: is the APACHE II score a reliable marker of physiological impairment?
 Your answer：APACHE II score is the most commonly used scoring system to evaluate severity of illness in the ICU. The APACHE II score is a 12-item scale that assesses the patient's age, comorbid conditions, and physiological variables (including respiratory rate, mean arterial pressure, heart rate, temperature, and Glasgow coma score). The APACHE II score is calculated by adding points for each of these variables. The APACHE II score ranges from 0 to 71, with higher scores indicating greater severity of illness. The APACHE II score has been shown to be a reliable marker of physiological impairment in critically ill patients.
 Your answer：
--------------------------------------------------
问题 (Key: 27281318): Can Flexible Instruments Create Adequate Femoral Tunnel L